In [3]:
import json
import urllib.request
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [43]:
urdata = pd.read_csv('C:/Users/mir96/문서/Code/DNFPJ/2. API 데이터 추출/2.1 고유코드_추출/2.1_final_result.csv', encoding = 'euc-kr')
urdata = urdata.dropna()

user_info = pd.DataFrame(columns = ['urname','adventurename', 'adventurelevel', 'resist', 'physical_defense', 'magical_defense'])

def none_data(urdata, user_info) :
    user_status = pd.DataFrame({
                    'urname' : [urname], 
                    'adventurename' : None,
                    'adventurelevel' : None,
                    'resist' : None,
                    'physical_defense' : None,
                    'magical_defense' : None
                    })
    user_info = pd.concat([user_info, user_status])
    return user_info


for i in range (0, len(urdata)) :
    urname = urdata.iloc[i,0]
    svname = urdata.iloc[i,1]
    chid = urdata.iloc[i,3]
    processpoint = (i+1)/len(urdata)
    requestData = requests.get(f'https://api.neople.co.kr/df/servers/{svname}/characters/{chid}/status?apikey=M23RMaIB2JlKPyRWYRBKDb6UtE3NDTrS')
    if requestData.status_code == 200 :
        jsonData = requestData.json()
        api_result = pd.json_normalize(jsonData)
        if api_result.empty :
            none_data(urname,user_info)
        elif api_result.loc[0, 'adventureName'] == None :
            none_data(urname,user_info)
        elif api_result.loc[0, 'buff'] == [] :
            none_data(urname,user_info)
        elif api_result.loc[0, 'level'] == '110':
            # 모험단명 추출
            adventurename = api_result.loc[0,'adventureName']

            # Status 항목 추출
            status = pd.json_normalize(jsonData, record_path = 'status')
            status = status.set_index('name')
            total_resist = status.loc['화속성 저항', 'value'] + status.loc['수속성 저항', 'value'] + status.loc['명속성 저항', 'value'] + status.loc['암속성 저항', 'value']
            physical_def = status.loc['물리 방어율', 'value']
            magical_def = status.loc['마법 방어율', 'value']   

            #Buff 항목 추출
            buff = pd.json_normalize(jsonData, record_path = 'buff')
            buff = buff.set_index('name')
            adventureLevel = buff.loc['모험단 버프' , 'level']
            user_status = pd.DataFrame({'urname' : [urname], 
                    'adventurename' : [adventurename],
                    'adventurelevel' : [adventureLevel],
                    'resist' : [total_resist],
                    'physical_defense' : [physical_def],
                    'magical_defense' : [magical_def]})
            user_info = pd.concat([user_info, user_status])
            print("%0.5f%%, %s번째 확인중입니다"%(processpoint, i+1))
        else :
            none_data(urname,user_info)
            user_info = pd.concat([user_info, user_status])
            print("%0.5f%%, %s번째 확인중입니다"%(processpoint, i+1))
    else :
        none_data(urname,user_info)
        user_info = pd.concat([user_info, user_status])
        print("%0.5f%%, %s번째 확인중입니다"%(processpoint, i+1))

total_user_status = pd.merge(urdata, user_info, on = 'urname')
total_user_status.to_csv('2.2_Status_result.csv' , encoding = 'euc-kr')
print('완료했습니다.')

0.00000%, 1번째 확인중입니다
0.00001%, 2번째 확인중입니다
0.00001%, 3번째 확인중입니다
0.00002%, 4번째 확인중입니다
0.00002%, 5번째 확인중입니다
0.00003%, 6번째 확인중입니다
0.00003%, 7번째 확인중입니다
0.00004%, 8번째 확인중입니다
0.00004%, 9번째 확인중입니다
0.00005%, 10번째 확인중입니다
0.00005%, 11번째 확인중입니다
0.00006%, 12번째 확인중입니다
0.00006%, 13번째 확인중입니다
0.00006%, 14번째 확인중입니다
0.00007%, 15번째 확인중입니다
0.00007%, 16번째 확인중입니다
0.00008%, 17번째 확인중입니다
0.00008%, 18번째 확인중입니다
0.00009%, 19번째 확인중입니다
0.00010%, 21번째 확인중입니다
0.00010%, 22번째 확인중입니다
0.00011%, 23번째 확인중입니다
0.00011%, 24번째 확인중입니다
0.00012%, 25번째 확인중입니다
0.00012%, 26번째 확인중입니다
0.00012%, 27번째 확인중입니다
0.00013%, 28번째 확인중입니다
0.00013%, 29번째 확인중입니다
0.00014%, 30번째 확인중입니다
0.00014%, 31번째 확인중입니다
0.00015%, 32번째 확인중입니다
0.00015%, 33번째 확인중입니다
0.00016%, 34번째 확인중입니다
0.00016%, 35번째 확인중입니다
0.00017%, 36번째 확인중입니다
0.00017%, 37번째 확인중입니다
0.00018%, 38번째 확인중입니다
0.00018%, 39번째 확인중입니다
0.00018%, 40번째 확인중입니다
0.00019%, 41번째 확인중입니다
0.00019%, 42번째 확인중입니다
0.00020%, 43번째 확인중입니다
0.00020%, 44번째 확인중입니다
0.00021%, 45번째 확인중입니다
0.00021%, 46번째 확인중입니다
0.00022%, 47번째 확인중입

KeyboardInterrupt: 

In [27]:
api_result

,characterId,characterName,level,jobId,jobGrowId,jobName,jobGrowName,adventureName,guildId,guildName,buff,status
0,537e52678c06a8cc18a4f2977c30b88d,Smdljhstd,84,986c2b3d72ee0e4a0b7fcfbe786d4e02,4ec01f4ae3728c080f28a72213b6df10,총검사,빅보스,됐다고평가,None,None,"[{'name': '모험단 버프', 'level': 33, 'status': [{'...","[{'name': 'HP', 'value': 34986}, {'name': 'MP'..."


In [16]:
api_result.loc[0, 'buff']

[]

In [32]:
if api_result.loc[0, 'level'] == '110':
# 모험단명 추출
    adventurename = api_result.loc[0,'adventureName']
    print(adventurename)
else :
    print(api_result.loc[0, 'level'])

84
